In [ ]:
# from def_all

%%time
####### 폴더 추가해서 복사하기 ####################

## 테두리가 만들어진 이미지 폴더들을 순화하며 폴더 명에 따라 회전 시키기

## 순환이 문자로(j) 인식되기 때문에 0, 1, 10, 100, 101, 102, 103... 109, 11, 110.. 순으로 작업 수행

import imgaug as ia
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

ia.seed(1)

# dir = 'D:/coding/data_set/lomin/train_copy/train_angle/' # train 테스트셋
# dir = 'D:/coding/data_set/lomin/train_copy/val_angle/' # val 테스트셋

dir = 'D:/coding/data_set/lomin/temp/val_angle/' # val 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    # os.makedirs(dir_d + '/dgr_' + j) 
    dir_2 = dir_dgr = dir_d + '/dgr_' + j + '/'
    

    images, annotations = read_train_dataset(dir_d)

    for idx in range(len(images)):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Sequential([
            # iaa.Multiply((1.2, 1.5)),
            iaa.Affine(
                # translate_px={"x": 40, "y": 60},
                # scale=(0.5, 0.7),
                rotate = int(j)
            )
        ])

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        # new_image_file = dir + 'after_' + annotations[idx][2]
        # new_image_file = dir_d + annotations[idx][2]
        new_image_file = dir_dgr + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            # name (label) , x1, x2, y1, y2 값 바꿈
            voc_writer.addObject(j), int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        # voc_writer.save(dir + 'after_' + annotations[idx][1])
        # voc_writer.save(dir_d + annotations[idx][1])
        voc_writer.save(dir_dgr + annotations[idx][1])
